jupyter로 flask 사용시 에러발생, 해결이 복잡하니 gedit으로 .py생성 후 실행 바람.

In [1]:
# -*- coding: utf-8 -*-
"""
 $ export FLASK_APP=urban_sound_classifier.py
 $ flask run
 ... Running on http://127.0.0.1:5000
"""

'\n $ export FLASK_APP=urban_sound_classifier.py\n $ flask run\n ... Running on http://127.0.0.1:5000\n'

In [2]:
import numpy as np
import tensorflow as tf
import librosa
from flask import Flask, request
import os

In [3]:
tf.reset_default_graph ()
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

In [4]:
training_epochs = 100
n_dim = 193
n_classes = 3
learning_rate = 0.001

In [5]:
X = tf.placeholder(tf.float32, [None, n_dim])
Y = tf.placeholder(tf.float32, [None, n_classes])

In [6]:
c1 = tf.layers.conv2d(tf.reshape(X, [-1, 1, n_dim, 1]), 50, (1, 5), padding='same', activation=tf.nn.sigmoid, name="c1")
p1 = tf.layers.max_pooling2d(inputs=c1, pool_size=[1, 2], strides=2)
c2 = tf.layers.conv2d(tf.reshape(p1, [-1, 1, 96, 50]), 100, (1, 5), padding='same', activation=tf.nn.sigmoid, name="c2")
p2 = tf.layers.max_pooling2d(inputs=c2, pool_size=[1, 2], strides=2)

h_p = tf.reshape(p2, [-1, 48*100])

h_1 = tf.layers.dense(inputs=h_p, units=4800, activation=tf.nn.sigmoid, kernel_initializer=tf.contrib.layers.xavier_initializer(), name="fc1")

y_hat = tf.layers.dense(inputs=h_1, units=n_classes, kernel_initializer=tf.contrib.layers.xavier_initializer(), name="h4")

y_sigmoid = tf.nn.sigmoid(y_hat)
y_ = tf.nn.softmax(y_hat)

init = tf.global_variables_initializer()

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.


In [7]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.restore(sess, 'model_adam.ckpt')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model_adam.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key c1/bias not found in checkpoint
	 [[node save/RestoreV2 (defined at <ipython-input-7-9a018f1d0e1b>:1) ]]

Original stack trace for 'save/RestoreV2':
  File "/home/monn/anaconda3/envs/356/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/monn/anaconda3/envs/356/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/monn/anaconda3/envs/356/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/monn/anaconda3/envs/356/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/monn/anaconda3/envs/356/lib/python3.5/site-packages/ipykernel/k

In [54]:
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = './upload'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

In [55]:
@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return ''
    file = request.files['file']
    if file.filename == '':
        return ''
    audio_file = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(audio_file)
    mfccs, chroma, mel, contrast,tonnetz = extract_feature(audio_file)
    x_data = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
    y_hat, sigmoid = sess.run([y_, y_sigmoid], feed_dict={X: x_data.reshape(1,-1)})
    index = np.argmax(y_hat)
    print(sigmoid)
    return '%d' % (index)